# Malha H3

### 1. Exportar o H3 para .csv

In [ ]:
import pandas as pd
import h3
import logging

logging.basicConfig(level=logging.INFO, format='%(message)s')

def preparar_csv_upload_com_setor():
    path_input = '../data/clean/h3/br/br_h3_res9_com_setores.parquet'
    # Mudei o nome levemente para você saber que é o novo
    output_csv = '../data/clean/h3/br/br_h3_para_upload_gee.csv' 

    logging.info("1. Lendo arquivo Parquet...")
    df = pd.read_parquet(path_input)

    logging.info("2. Gerando coordenadas e colunas auxiliares...")
    
    # Gerar Lat/Lon (h3 ver. 4+)
    coords = df['h3_id'].apply(h3.cell_to_latlng) 
    df['latitude'] = [x[0] for x in coords]
    df['longitude'] = [x[1] for x in coords]

    # Garantir que colunas de ID sejam strings (texto)
    df['CD_MUN'] = df['CD_MUN'].astype(str)
    df['CD_UF'] = df['CD_MUN'].str[:2]
    
    # --- NOVO: Garantir CD_SETOR ---
    # Verifica se a coluna existe e converte para string
    if 'CD_SETOR' in df.columns:
        df['CD_SETOR'] = df['CD_SETOR'].astype(str)
    else:
        logging.warning("AVISO: Coluna 'CD_SETOR' não encontrada no arquivo!")

    # Selecionar colunas para exportação (Incluindo CD_SETOR agora)
    cols_export = ['h3_id', 'latitude', 'longitude', 'CD_UF', 'CD_MUN', 'CD_SETOR']
    
    logging.info(f"3. Salvando {output_csv}...")
    df[cols_export].to_csv(output_csv, index=False)
    logging.info("Concluído! Pode fazer o upload no GEE.")

preparar_csv_upload_com_setor()

1. Lendo arquivo Parquet...
2. Gerando coordenadas e colunas auxiliares...
3. Salvando ../data/clean/h3/br/br_h3_cdmun_para_upload_gee.csv...
Concluído! Pode fazer o upload no GEE.


### 2. Aplicar no Earth Engine Code Editor

```javascript
// --- 1. CONFIGURAÇÃO ---
var table = ee.FeatureCollection("projects/ee2-linafaccin/assets/br_h3_cdmun_para_upload_gee");

// Lista de UFs
var ufs = [
  '11','12','13','14','15','16','17', 
  '21','22','23','24','25','26','27','28','29', 
  '31','32','33','35', 
  '41','42','43', 
  '50','51','52','53' 
];

// --- 2. SATÉLITE COM REMOÇÃO DE NUVENS ---
function prepL8(image) {
  var qa = image.select('QA_PIXEL');
  var mask = qa.bitwiseAnd(1 << 3).eq(0).and(qa.bitwiseAnd(1 << 4).eq(0));
  return image.select('ST_B10')
    .updateMask(mask)
    .multiply(0.00341802).add(149.0).subtract(273.15)
    .rename('temp_media_solo') // Renomeia a banda, mas o reducer precisa saber disso tb
    .copyProperties(image, ['system:time_start']);
}

var landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterDate('2014-01-01', '2024-12-31')
    .map(prepL8);

var lst_mean = landsat8.mean();

// --- 3. PROCESSAMENTO E EXPORTAÇÃO ---
ufs.forEach(function(uf_code) {
  
  var uf_collection = table.filter(ee.Filter.eq('CD_UF', uf_code));

  var h3_with_temp = lst_mean.reduceRegions({
    collection: uf_collection,
    // --- CORREÇÃO: Forçar o nome da saída ---
    reducer: ee.Reducer.mean().setOutputs(['temp_media_solo']), 
    scale: 30,
    tileScale: 16
  });

  // Exporta
  Export.table.toDrive({
    collection: h3_with_temp,
    description: 'H3_LST_UF_' + uf_code,
    folder: 'GEE_Resultados_H3_Centroides',
    fileFormat: 'CSV',
    selectors: ['h3_id', 'CD_UF', 'CD_MUN', 'CD_SETOR', 'temp_media_solo']
  });
});

print("Tarefas enviadas! Agora os nomes das colunas vão bater.");
```

### 3. Juntar arquivos csv do GEE

In [4]:
import pandas as pd
import glob
import logging
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def finalizar_processamento_h3():
    # Caminhos
    path_csv_folder = '../data/raw/gee/land_surface_temperature/uf/' 
    path_output_parquet = '../data/clean/h3/br/br_h3_res9_e4_calor.parquet'
    
    all_files = glob.glob(os.path.join(path_csv_folder, "*.csv"))
    
    if not all_files:
        logging.error("Nenhum arquivo CSV encontrado!")
        return

    logging.info(f"Encontrados {len(all_files)} arquivos. Iniciando leitura...")

    df_list = []
    for filename in all_files:
        # Lê garantindo que IDs sejam texto
        df = pd.read_csv(filename, dtype={'CD_SETOR': str, 'CD_MUN': str, 'CD_UF': str})
        df_list.append(df)

    # 1. Consolidação
    df_final = pd.concat(df_list, ignore_index=True)
    
    # [MELHORIA 1] Remover duplicatas de H3_ID (segurança contra sobreposição de arquivos)
    total_antes = len(df_final)
    df_final = df_final.drop_duplicates(subset=['h3_id'])
    logging.info(f"Total carregado: {total_antes} | Após remover duplicatas: {len(df_final)}")

    # [MELHORIA 2] Limpar colunas desnecessárias do GEE
    cols_drop = ['system:index', '.geo']
    df_final = df_final.drop(columns=[c for c in df_final.columns if c in cols_drop])

    # 2. Tratamento de Nulos (Estatística Local)
    nulos = df_final['temp_media_solo'].isnull().sum()
    if nulos > 0:
        logging.warning(f"Encontrados {nulos} valores nulos. Tentando imputação...")
        
        # Tenta preencher com a média DO MUNICÍPIO primeiro (mais preciso)
        df_final['temp_media_solo'] = df_final['temp_media_solo'].fillna(
            df_final.groupby('CD_MUN')['temp_media_solo'].transform('mean')
        )
        
        # Se ainda sobrar nulo (município inteiro sem dados), usa média geral
        media_geral = df_final['temp_media_solo'].mean()
        df_final['temp_media_solo'] = df_final['temp_media_solo'].fillna(media_geral)

    # 3. Normalização (0 a 1)
    # [DICA] Verificar se há outliers absurdos (ex: vulcão ou erro de sensor > 60 graus ou < -10)
    # Aqui assumimos que a média de 10 anos suavizou isso.
    t_min = df_final['temp_media_solo'].min()
    t_max = df_final['temp_media_solo'].max()
    logging.info(f"Temp Mínima: {t_min:.2f}°C | Temp Máxima: {t_max:.2f}°C")
    
    df_final['e4_cal_norm'] = (df_final['temp_media_solo'] - t_min) / (t_max - t_min)

    # 4. Salvar
    df_final.to_parquet(path_output_parquet, compression='snappy') # Snappy é rápido e padrão
    logging.info(f"SUCESSO! Arquivo salvo em: {path_output_parquet}")

if __name__ == "__main__":
    finalizar_processamento_h3()

Encontrados 27 arquivos. Iniciando leitura...
Total carregado: 4892991 | Após remover duplicatas: 4892991
Encontrados 265116 valores nulos. Tentando imputação...
Temp Mínima: 3.99°C | Temp Máxima: 74.45°C
SUCESSO! Arquivo salvo em: ../data/clean/h3/br/br_h3_res9_e4_calor.parquet


---

# Setores Censitários

### 1. Criar shp das UFs e converter para zip, para fazer upload no GEE

In [ ]:
import geopandas as gpd
import logging
import os
import zipfile

# Configuração de Logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def preparar_setores_para_gee():
    # Converta para caminho absoluto para evitar erros de "../"
    input_path = os.path.abspath('../data/raw/ibge/setores_censitarios_cd2022/BR_setores_CD2022.gpkg')
    output_dir = os.path.abspath('../data/raw/ibge/setores_censitarios_cd2022/gee_por_uf/')
    
    # Cria a pasta se ela não existir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        logging.info(f"Pasta criada: {output_dir}")

    logging.info("1. Lendo arquivo de setores...")
    gdf = gpd.read_file(input_path)
    
    logging.info("2. Simplificando geometrias...")
    gdf['geometry'] = gdf.geometry.simplify(tolerance=0.0001, preserve_topology=True)
    
    logging.info("3. Iniciando exportação por UF...")
    
    gdf['CD_UF'] = gdf['CD_UF'].astype(str)
    gdf_export = gdf[['CD_SETOR', 'CD_UF', 'CD_MUN', 'geometry']]
    
    ufs = gdf_export['CD_UF'].unique()
    
    for uf in ufs:
        gdf_uf = gdf_export[gdf_export['CD_UF'] == uf]
        
        # Nome base do arquivo
        base_name = f"br_setores_cd2022_{uf}_gee"
        
        # 1. Salvar como Shapefile NORMAL (não zipado ainda)
        shp_path = os.path.join(output_dir, f"{base_name}.shp")
        logging.info(f"   Processando UF {uf}...")
        
        try:
            gdf_uf.to_file(shp_path, driver='ESRI Shapefile', encoding='utf-8')
            
            # 2. Zipar manualmente os arquivos gerados (.shp, .shx, .dbf, .prj, .cpg)
            zip_path = os.path.join(output_dir, f"{base_name}.zip")
            
            with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                # O Shapefile gera vários arquivos com o mesmo nome e extensões diferentes
                extensions = ['.shp', '.shx', '.dbf', '.prj', '.cpg']
                for ext in extensions:
                    file_part = os.path.join(output_dir, base_name + ext)
                    if os.path.exists(file_part):
                        # Adiciona ao zip (arcname define o nome dentro do zip, sem pastas)
                        zipf.write(file_part, arcname=base_name + ext)
                        # Opcional: Apagar o arquivo solto depois de zipar
                        os.remove(file_part)
            
            logging.info(f"   -> Zip criado com sucesso: {zip_path}")
            
        except Exception as e:
            logging.error(f"   Erro ao processar UF {uf}: {str(e)}")

    logging.info("Pronto! Todos os arquivos .zip foram gerados.")

if __name__ == "__main__":
    preparar_setores_para_gee()

1. Lendo arquivo de setores...


2. Simplificando geometrias...
3. Iniciando exportação por UF...
   Processando UF 11...
Created 3,456 records
   -> Zip criado com sucesso: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute\Carolina WRI\Repositorios\climate-justice-index\data\raw\ibge\setores_censitarios_cd2022\gee_por_uf\br_setores_cd2022_11_gee.zip
   Processando UF 12...
Created 2,215 records
   -> Zip criado com sucesso: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute\Carolina WRI\Repositorios\climate-justice-index\data\raw\ibge\setores_censitarios_cd2022\gee_por_uf\br_setores_cd2022_12_gee.zip
   Processando UF 13...
Created 11,755 records
   -> Zip criado com sucesso: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute\Carolina WRI\Repositorios\climate-justice-index\data\raw\ibge\setores_censitarios_cd2022\gee_por_uf\br_setores_cd2022_13_gee.zip
   Processando UF 14...
Created 1,986 records
   -> Zip criado com sucesso: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute

### 2. Gerar a temperatura superficial média por setor censitário no Earth Engine Code Editor

```javascript
// --- 1. CONFIGURAÇÃO ---

// Definimos a "raiz" do nome do seu asset
var asset_base = "projects/ee2-linafaccin/assets/br_setores_cd2022_";
var asset_suffix = "_gee";

// Lista de UFs (Strings)
var ufs = [
  '11','12','13','14','15','16','17', // Norte
  '21','22','23','24','25','26','27','28','29', // Nordeste
  '31','32','33','35', // Sudeste
  '41','42','43', // Sul
  '50','51','52','53' // Centro-Oeste
];

// --- 2. PREPARAR SATÉLITE (Igual ao anterior) ---
function prepL8(image) {
  var qa = image.select('QA_PIXEL');
  // Mascara nuvens e sombras (Bits 3 e 4)
  var mask = qa.bitwiseAnd(1 << 3).eq(0).and(qa.bitwiseAnd(1 << 4).eq(0));
  return image.select('ST_B10')
    .updateMask(mask)
    .multiply(0.00341802).add(149.0).subtract(273.15)
    .rename('temp_media_solo')
    .copyProperties(image, ['system:time_start']);
}

// Carrega a coleção Landsat 8 (2014-2024)
var landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterDate('2014-01-01', '2024-12-31')
    .map(prepL8);

// Calcula a média de todo o período em uma única imagem
var lst_mean = landsat8.mean();

// --- 3. PROCESSAMENTO (LOOP DINÂMICO) ---
ufs.forEach(function(uf_code) {
  
  // AQUI ESTÁ A MUDANÇA: Construímos o ID do asset para esta UF específica
  // Ex: projects/ee2-linafaccin/assets/br_setores_cd2022_24_gee
  var asset_id = asset_base + uf_code + asset_suffix;
  
  // Carregamos apenas o estado da vez
  var uf_collection = ee.FeatureCollection(asset_id);

  // reduceRegions: Pega a média dos pixels dentro de cada polígono (Setor)
  var setores_com_temp = lst_mean.reduceRegions({
    collection: uf_collection,
    reducer: ee.Reducer.mean().setOutputs(['temp_media_solo']), 
    scale: 30,
    tileScale: 16 // Essencial para polígonos complexos não darem erro de memória
  });

  // Limpeza: Seleciona colunas e remove geometria para exportação leve
  var export_collection = setores_com_temp.select(
      ['CD_SETOR', 'CD_MUN', 'CD_UF', 'temp_media_solo'], 
      null, 
      false 
  );

  // Exporta
  Export.table.toDrive({
    collection: export_collection,
    description: 'Setores_LST_UF_' + uf_code,
    folder: 'GEE_Setores_Temp_Final', // Sugiro uma pasta nova para não misturar com os hexágonos
    fileFormat: 'CSV',
    selectors: ['CD_SETOR', 'CD_MUN', 'CD_UF', 'temp_media_solo']
  });
});

print("Tarefas enviadas! Verifique a aba Tasks.");
```

### 3. Unir os arquivos .csv e salvar em .gpkg

In [11]:
import pandas as pd
import geopandas as gpd
import glob
import logging
import os
import numpy as np

# Configuração de Logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def processar_temperatura_setores():
    # --- CAMINHOS ---
    # Pasta onde estão os CSVs baixados do GEE
    path_csv_folder = '../data/raw/gee/land_surface_temperature/sc_uf/'
    
    # Arquivo GPKG de entrada (O arquivo que já tem os índices e1, e2, e3, e5)
    path_gpkg_input = '../data/clean/sc/gpkg/BR_setores_CD2022_e1235_vul_int.gpkg'
    
    # Arquivos de SAÍDA (GeoPackage e CSV)
    path_gpkg_output = '../data/clean/sc/gpkg/BR_setores_CD2022_e12345_vul_int.gpkg'
    path_csv_output  = '../data/clean/sc/csv/BR_setores_CD2022_e12345_vul_int.csv'

    # Garante que as pastas de saída existem
    os.makedirs(os.path.dirname(path_gpkg_output), exist_ok=True)
    os.makedirs(os.path.dirname(path_csv_output), exist_ok=True)

    # --- 1. LER E CONSOLIDAR OS CSVS DO GEE ---
    logging.info("1. Buscando arquivos CSV de temperatura...")
    all_files = glob.glob(os.path.join(path_csv_folder, "*.csv"))
    
    if not all_files:
        logging.error("Nenhum CSV encontrado! Verifique o caminho.")
        return

    logging.info(f"   Encontrados {len(all_files)} arquivos. Lendo...")
    
    df_list = []
    for filename in all_files:
        # Importante: Ler CD_SETOR como string para não perder precisão ou zeros
        df = pd.read_csv(filename, dtype={'CD_SETOR': str, 'CD_MUN': str, 'CD_UF': str})
        df_list.append(df)

    # Junta todos os estados em um único DataFrame
    df_temp = pd.concat(df_list, ignore_index=True)
    
    # Remove colunas inúteis do GEE se existirem
    cols_drop = ['system:index', '.geo']
    df_temp = df_temp.drop(columns=[c for c in df_temp.columns if c in cols_drop])
    
    # Remove duplicatas (prevenção)
    df_temp = df_temp.drop_duplicates(subset=['CD_SETOR'])
    
    logging.info(f"   Total de setores com temperatura recuperados: {len(df_temp)}")

    # --- 2. LER O GEOPACKAGE ORIGINAL ---
    logging.info(f"2. Lendo arquivo GeoPackage de entrada: {path_gpkg_input}")
    # Lê o arquivo que contém os dados anteriores
    gdf_setores = gpd.read_file(path_gpkg_input)
    
    # Garantir que a chave de cruzamento seja string
    gdf_setores['CD_SETOR'] = gdf_setores['CD_SETOR'].astype(str)

    # --- 3. MERGE (CRUZAMENTO) ---
    logging.info("3. Realizando o Join (GeoPackage + Dados de Temperatura)...")
    
    # Selecionamos apenas as colunas de interesse do CSV
    cols_to_merge = ['CD_SETOR', 'temp_media_solo']
    
    # Left Join: Mantém todos os setores do mapa
    gdf_final = gdf_setores.merge(df_temp[cols_to_merge], on='CD_SETOR', how='left')

    # --- 4. TRATAMENTO DE NULOS (IMPUTAÇÃO) ---
    nulos = gdf_final['temp_media_solo'].isnull().sum()
    if nulos > 0:
        logging.warning(f"   Encontrados {nulos} setores sem dados de temperatura.")
        logging.info("   Realizando imputação pela média do município...")
        
        # Garante que temos CD_MUN
        if 'CD_MUN' not in gdf_final.columns:
            gdf_final['CD_MUN'] = gdf_final['CD_SETOR'].str[:7]

        # 1. Média do MUNICÍPIO
        gdf_final['temp_media_solo'] = gdf_final['temp_media_solo'].fillna(
            gdf_final.groupby('CD_MUN')['temp_media_solo'].transform('mean')
        )
        
        # 2. Fallback: Média GERAL
        media_geral = gdf_final['temp_media_solo'].mean()
        gdf_final['temp_media_solo'] = gdf_final['temp_media_solo'].fillna(media_geral)
        
        restantes = gdf_final['temp_media_solo'].isnull().sum()
        logging.info(f"   Nulos restantes após imputação: {restantes}")

    # --- 5. NORMALIZAÇÃO (0 a 1) ---
    logging.info("5. Calculando normalização 'e4_cal_norm'...")
    
    t_min = gdf_final['temp_media_solo'].min()
    t_max = gdf_final['temp_media_solo'].max()
    
    logging.info(f"   Temp Mínima: {t_min:.2f}°C | Temp Máxima: {t_max:.2f}°C")
    
    # Fórmula Min-Max
    gdf_final['e4_cal_norm'] = (gdf_final['temp_media_solo'] - t_min) / (t_max - t_min)

    # --- 6. SALVAR ARQUIVOS ---
    logging.info(f"6. Salvando arquivo GeoPackage: {path_gpkg_output}")
    gdf_final.to_file(path_gpkg_output, driver='GPKG')
    
    logging.info(f"7. Salvando arquivo CSV: {path_csv_output}")
    # Removemos a geometria para o CSV ficar leve e não dar erro de tamanho
    df_csv = pd.DataFrame(gdf_final.drop(columns='geometry'))
    df_csv.to_csv(path_csv_output, index=False)
    
    logging.info("PROCESSO CONCLUÍDO COM SUCESSO!")

if __name__ == "__main__":
    processar_temperatura_setores()

1. Buscando arquivos CSV de temperatura...
   Encontrados 27 arquivos. Lendo...
   Total de setores com temperatura recuperados: 468099
2. Lendo arquivo GeoPackage de entrada: ../data/clean/sc/gpkg/BR_setores_CD2022_e1235_vul_int.gpkg
3. Realizando o Join (GeoPackage + Dados de Temperatura)...
   Encontrados 4024 setores sem dados de temperatura.
   Realizando imputação pela média do município...
   Nulos restantes após imputação: 0
5. Calculando normalização 'e4_cal_norm'...
   Temp Mínima: 11.25°C | Temp Máxima: 63.49°C
6. Salvando arquivo GeoPackage: ../data/clean/sc/gpkg/BR_setores_CD2022_e12345_vul_int.gpkg
Created 472,780 records
7. Salvando arquivo CSV: ../data/clean/sc/csv/BR_setores_CD2022_e12345_vul_int.csv
PROCESSO CONCLUÍDO COM SUCESSO!
